In [3]:
"""
글의 키워드를 뽑아서 서로 다른 글들의 유사도를 판별하였다.

글 A에서 가장 많이 나온 명사들 10개를 뽑고
그중 2글자 이상인것들만 따와서

글이 너무 많아서 특정 카테고리의 글을 쓸 때
비슷한 글을 찾아는 방식으로 구현했다.

특히 2017-11-11 이명박 관련글이 제일 중복이 많아서
보여주기에 알맞다.

"""

'\n글의 키워드를 뽑아서 서로 다른 글들의 유사도를 판별하였다.\n\n글 A에서 가장 많이 나온 명사들 10개를 뽑고\n그중 2글자 이상인것들만 따와서\n\n글이 너무 많아서 특정 카테고리의 글을 쓸 때\n비슷한 글을 찾아는 방식으로 구현했다.\n\n\n'

In [4]:
import pandas as pd
import numpy as np
import re
print(pd.__version__)
print(np.__version__)

0.24.2
1.16.2


In [5]:
# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
petitions = pd.read_csv('petition.csv',
                        parse_dates=['start', 'end'])
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(377756, 8)

In [6]:
df=petitions

In [7]:
df.isnull().sum()

article_id    0
start         0
end           0
answered      0
votes         0
category      0
title         0
content       1
dtype: int64

In [8]:
df.shape

(377756, 8)

In [9]:
df.isnull().values.any()

True

In [10]:
df=df.dropna()

In [11]:
df.shape

(377755, 8)

In [12]:
# 전체 데이터 중 투표가 500건 이상인 데이터를 기준으로 가져옵니다. 
# 아웃라이어 데이터 제거를 위해 20만건 이상 데이터도 제거합니다.
petition_remove_outlier = df.loc[(df['category']=='정치개혁')]
petition_remove_outlier.shape

(59019, 8)

In [13]:
df = petition_remove_outlier.copy()

In [14]:
df

,article_id,start,end,answered,votes,category,title,content
6,27,2017-08-19,2017-09-18,0,1,정치개혁,김이수 헌재소장 임명 재고 건의,문 대통령님께 묻습니다 (2017. 8. 19)\n저는 경기도 용인에 살고 있는 국...
96,120,2017-08-20,2017-11-18,0,20,정치개혁,여론조작 처벌 도입과 무거운 형량을 원합니다,달빛기사단 문꿀오소리 이런 집단이 좌표찍고 자신들 입맛에 맞지않는 기사 댓글들을 비...
156,180,2017-08-21,2017-11-19,0,5,정치개혁,내 제안 건의를 대통령이 직접 답변해 주세요.,"1. 대통령이 직접 답변준다고 해서 그대로 믿었습니다.\n그런데, 접수대기로 접수조..."
225,250,2017-08-22,2017-11-20,0,11,정치개혁,청소년 참정권을 보장해주십시오!,청소년 참정권을 보장해주십시오!\n현재 대한민국은 세계 95% 가까운 국가에서 선거...
257,282,2017-08-22,2017-11-20,0,1,정치개혁,정부의 정책의 명확한 기준을 만들어주세요,"안녕하세요.\n밝은 미래를 바라보며, 살아가는 대한민국의 국민입니다.\n새정부의 정..."
368,395,2017-08-24,2017-08-31,0,0,정치개혁,"만 18세 투표권, 만 19세 정치 출마 제안",안녕하세요! 저는 서울에 위치한 고등학생입니다.\n저는 만 18세 투표권을 제안하고...
406,434,2017-08-24,2017-09-23,0,38,정치개혁,생존권 위협하는 집합건물법 개정이 필요합니다,1인가구가 늘어나고 있는 이 시대에 주거용 오피스텔 관련법인 집합건물법에는 문제가 ...
445,473,2017-08-24,2017-08-31,0,0,정치개혁,문재인 대통령 나으리... 청와대에 강도질 좀 해도 되겠습니까?,교통사고는 건강보험 수가체계를 따르도록 되어 있어\n합의를 하지 않으면 치료조차 받...
486,514,2017-08-25,2017-09-24,0,1563,정치개혁,세월호 참사의 주범인 해경 123정 해경대원 13명에 대한 정부 직접 조사를 청원합니다.,518 당시 헬기 사격 및 전투기 대기 명령에 대한 정부 차원의 특별조사를 지시한 ...
531,559,2017-08-25,2017-09-24,0,0,정치개혁,문재인 대통령님께 드리는 편지 및 이상적인 제안.,존경하는 문재인 대통령님께\n언제 대통령님께 편지를 써보나 하고 기다리고\n또 기다...


In [15]:
lines=df["content"][377720]
print(lines)

최순실 등의 국정농단으로 대한민국은 뿌리가 흔들렸었습니다.\n이로인해 전 대통령은 탄핵 후 감옥에 있고, 전전 대통령 또한 국정농단 수준의 범죄로 감옥에 있습니다.\n이게 나라입니까?\n문제인 정부는 최순실 국정농단으로 들어선 정부입니다.\n하지만 지금도 정치, 사회 등에 온갖 적폐세력이 독버섯처럼 숨죽이고 있는 것 같습니다.\n국민 혈세로 외유성 해외출장 가서 나라를 망신시키는 기초의회 의원도 있습니다.\n어제 밤에 눈과 귀를 의심하는 뉴스가 보도되었습니다.\n쇼트트렉 심석희 선수가 고2때부터 평창동계올림픽 개최 1개월 전까지 코치에게 성폭행을 당했다는 것입니다.\n대통령이 되고자 공당의 후보로 나왔던 인사도 비서를 성폭행했다 하여 재판을 받고 있습니다.\n이게 나라입니까?\n대한민국 방방곡곡에 적폐세력이 득실 거리고 있는 것 같습니다.\n국민은 안중에도 없고 본인들의 밥 그릇에만 관심 있는 정치는을 비롯하여.....\n대한민국의 체육계는 총체적 난국을 뛰어 넘었습니다.\n어디부터 손을 대야 하는 걸까요? 알 수 없습니다.\n초, 중학교 야구, 축구선수들은 코치 등에게 온갖 욕과 폭력에 시달리고, 학부모들은 내 자식이 피해를 입을까봐. 이러지도 저러지도 못합니다.\n또한 시합 기간, 전지훈련을 가게되면 코치 등은 왕이됩니다.\n이게 나라입니까?\n썩어문드러진 체육계를 어떻게 손 볼 수 있을까요?\n비단 심석희 선수만이 아닐 것이라고 생각합니다.\n이번 기회에 체육계에 독버섯처럼 숨죽이고 있는 독버섯의 뿌리를 뽑아줄 것을 국민의 한 사람으로서 강력하게 청원합니다.


In [16]:
from konlpy.tag import Twitter

nlpy = Twitter()

nouns = nlpy.nouns(lines)

print(nouns)

C:\Users\hyunho\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['최', '실', '등', '국정', '농단', '대한민국', '뿌리', '전', '대통령', '탄핵', '후', '감옥', '전전', '대통령', '또한', '국정', '농단', '수준', '범죄', '감옥', '이', '나라', '입', '문제', '정부', '최', '실', '국정', '농단', '정부', '지금', '정치', '사회', '등', '온갖', '적폐', '세력', '독버섯', '숨죽', '것', '국민', '혈', '세로', '외', '유성', '해외', '출장', '나라', '망신', '기초의회', '의원', '어제', '밤', '눈', '귀', '의심', '뉴스', '보도', '쇼트트렉', '심석희', '선수', '고', '때', '평창동계올림픽', '개최', '개월', '전', '코치', '성폭행', '것', '대통령', '공당', '후보', '인사', '비서', '성폭행', '재판', '이', '나라', '입', '대한민국', '방방곡곡', '적폐', '세력', '득실', '거리', '것', '국민', '안중', '본인', '밥', '그릇', '관심', '정치', '비롯', '대한민국', '체육', '체적', '난국', '어디', '손', '대야', '요', '알', '수', '초', '중학교', '야구', '축구선수', '코치', '등', '온갖', '욕', '폭력', '학부모', '내', '자식', '피해', '또한', '시합', '기간', '훈련', '가게', '코치', '등', '왕', '이', '나라', '입', '썩', '어문', '체육', '손', '볼', '수', '비단', '심석희', '선수', '것', '생각', '이번', '기회', '체육', '독버섯', '숨죽', '독버섯', '뿌리', '것', '국민', '사람', '청원']


In [18]:
from collections import Counter

count = Counter(nouns)



tag_count = []

tags = []



for n, c in count.most_common(10):
    dics = {'tag': n, 'count': c}
    if len(dics['tag']) >= 2 and len(tags) <= 49:
        tag_count.append(dics)
        tags.append(dics['tag'])

In [19]:
for n, c in count.most_common(10):
    print(n)

것
등
나라
국정
농단
대한민국
대통령
이
입
독버섯


In [20]:
for tag in tag_count:
    print(" {:<14}".format(tag['tag']), end='\t')
    print("{}".format(tag['count']))

 나라            	4
 국정            	3
 농단            	3
 대한민국          	3
 대통령           	3
 독버섯           	3


In [21]:
tag_count[0]

{'tag': '나라', 'count': 4}

In [22]:
tag

{'tag': '독버섯', 'count': 3}

In [24]:
df

,article_id,start,end,answered,votes,category,title,content
6,27,2017-08-19,2017-09-18,0,1,정치개혁,김이수 헌재소장 임명 재고 건의,문 대통령님께 묻습니다 (2017. 8. 19)\n저는 경기도 용인에 살고 있는 국...
96,120,2017-08-20,2017-11-18,0,20,정치개혁,여론조작 처벌 도입과 무거운 형량을 원합니다,달빛기사단 문꿀오소리 이런 집단이 좌표찍고 자신들 입맛에 맞지않는 기사 댓글들을 비...
156,180,2017-08-21,2017-11-19,0,5,정치개혁,내 제안 건의를 대통령이 직접 답변해 주세요.,"1. 대통령이 직접 답변준다고 해서 그대로 믿었습니다.\n그런데, 접수대기로 접수조..."
225,250,2017-08-22,2017-11-20,0,11,정치개혁,청소년 참정권을 보장해주십시오!,청소년 참정권을 보장해주십시오!\n현재 대한민국은 세계 95% 가까운 국가에서 선거...
257,282,2017-08-22,2017-11-20,0,1,정치개혁,정부의 정책의 명확한 기준을 만들어주세요,"안녕하세요.\n밝은 미래를 바라보며, 살아가는 대한민국의 국민입니다.\n새정부의 정..."
368,395,2017-08-24,2017-08-31,0,0,정치개혁,"만 18세 투표권, 만 19세 정치 출마 제안",안녕하세요! 저는 서울에 위치한 고등학생입니다.\n저는 만 18세 투표권을 제안하고...
406,434,2017-08-24,2017-09-23,0,38,정치개혁,생존권 위협하는 집합건물법 개정이 필요합니다,1인가구가 늘어나고 있는 이 시대에 주거용 오피스텔 관련법인 집합건물법에는 문제가 ...
445,473,2017-08-24,2017-08-31,0,0,정치개혁,문재인 대통령 나으리... 청와대에 강도질 좀 해도 되겠습니까?,교통사고는 건강보험 수가체계를 따르도록 되어 있어\n합의를 하지 않으면 치료조차 받...
486,514,2017-08-25,2017-09-24,0,1563,정치개혁,세월호 참사의 주범인 해경 123정 해경대원 13명에 대한 정부 직접 조사를 청원합니다.,518 당시 헬기 사격 및 전투기 대기 명령에 대한 정부 차원의 특별조사를 지시한 ...
531,559,2017-08-25,2017-09-24,0,0,정치개혁,문재인 대통령님께 드리는 편지 및 이상적인 제안.,존경하는 문재인 대통령님께\n언제 대통령님께 편지를 써보나 하고 기다리고\n또 기다...


In [25]:
df["10word"]=0

In [23]:
from collections import Counter

count = Counter(nouns)

In [27]:
from konlpy.tag import Twitter

nlpy = Twitter()

In [37]:
for index in df.index[0:5000]:
    nouns=0
    count=0
    lines=df["content"][index]
    nouns = nlpy.nouns(lines)
    count = Counter(nouns)
    list_10=[]
    for n,c in count.most_common(10):
        if len(n) >= 2 :
            list_10.append(n)
        
    df["10word"][index]=list_10

C:\Users\hyunho\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [88]:
#내가 쓴글
original_text=df["content"][33403]

nouns=0
count=0
lines=df["content"][index]
nouns = nlpy.nouns(lines)
count = Counter(nouns)
list_10=[]
for n,c in count.most_common(10):
    if len(n) >= 2 :
        list_10.append(n)        
original_10word=list_10

In [89]:
list_10

['이명박', '출국금지', '청원']

In [90]:
df["10word"][4500:4600]

33294     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33295     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33296     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33297     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33298     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33299                           [이명박, 출국, 금지]
33301     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33302     [국정원, 댓글, 사이버, 사령부, 정치, 개입, 관련, 조사]
33303                                  [출국금지]
33304                       [이명박, 즉시, 출국, 정지]
33305           [이번, 잠수, 가능성, 매우, 출국, 금지, 수사]
33306                           [이명박, 금지, 청원]
33309                  [증거, 지금, 시간, 인멸될, 가능성]
33310                           [이명박, 출국, 금비]
33311                           [이명박, 출구, 금지]
33312                             [이명박, 출국금지]
33313                         [이명박, 출국금지, 청원]
33314                          [헌법, 출국금지, 요청]
33315                             [이명박, 출국금지]
33317                             [이명박, 출국금지]
33318                             [이명박, 출국금지]
33319                         [이명박

In [91]:
df["10word"][33403]

['이명박', '대통령', '출국금지', '청원', '도피', '출국', '의도']

In [92]:
df["10word"][2695]

['청소년', '사건', '보호', '인천', '초등학생', '부산', '여중생', '이유']

In [93]:
similar_dict={}
original_index=33403
original_10word=df["10word"][original_index]
for index in df.index[0:5000]:
    sum=0
    for word1 in df["10word"][index]:
        for word2 in original_10word:
            if word1==word2:
                sum=sum+1
    sum=sum/(len(df["10word"][index])+len(original_10word)-sum)*100
    if (sum<50)&(sum>20)&(sum!=100) :
        similar_dict[index]=sum

In [94]:
similar_dict

{14494: 25.0,
 14497: 25.0,
 14526: 25.0,
 14529: 25.0,
 14532: 25.0,
 19948: 21.428571428571427,
 30030: 25.0,
 30037: 22.22222222222222,
 30043: 25.0,
 30044: 28.57142857142857,
 30055: 28.57142857142857,
 30056: 22.22222222222222,
 30059: 22.22222222222222,
 30063: 42.857142857142854,
 30069: 30.0,
 30072: 28.57142857142857,
 30079: 28.57142857142857,
 30086: 28.57142857142857,
 30088: 42.857142857142854,
 30089: 25.0,
 30091: 25.0,
 30097: 28.57142857142857,
 30099: 28.57142857142857,
 30104: 25.0,
 30110: 42.857142857142854,
 30112: 22.22222222222222,
 30117: 42.857142857142854,
 30118: 22.22222222222222,
 30121: 25.0,
 30131: 22.22222222222222,
 30135: 30.0,
 30139: 37.5,
 30142: 28.57142857142857,
 30156: 28.57142857142857,
 30158: 42.857142857142854,
 30159: 28.57142857142857,
 30166: 28.57142857142857,
 30174: 42.857142857142854,
 30179: 28.57142857142857,
 30183: 25.0,
 30185: 28.57142857142857,
 30197: 28.57142857142857,
 30209: 22.22222222222222,
 30213: 28.57142857142857,


In [95]:
import operator
similar_dict=sorted(similar_dict.items(), key=operator.itemgetter(1),reverse=True)

In [86]:
similar_list=similar_dict
similar_list

[(30320, 45.45454545454545),
 (32403, 45.45454545454545),
 (31107, 44.44444444444444),
 (31732, 44.44444444444444),
 (31998, 44.44444444444444),
 (33414, 44.44444444444444),
 (30063, 42.857142857142854),
 (30088, 42.857142857142854),
 (30110, 42.857142857142854),
 (30117, 42.857142857142854),
 (30158, 42.857142857142854),
 (30174, 42.857142857142854),
 (30277, 42.857142857142854),
 (30278, 42.857142857142854),
 (30283, 42.857142857142854),
 (30290, 42.857142857142854),
 (30292, 42.857142857142854),
 (30308, 42.857142857142854),
 (30314, 42.857142857142854),
 (30319, 42.857142857142854),
 (30325, 42.857142857142854),
 (30353, 42.857142857142854),
 (30376, 42.857142857142854),
 (30395, 42.857142857142854),
 (30400, 42.857142857142854),
 (30405, 42.857142857142854),
 (30423, 42.857142857142854),
 (30462, 42.857142857142854),
 (30472, 42.857142857142854),
 (30609, 42.857142857142854),
 (30619, 42.857142857142854),
 (30623, 42.857142857142854),
 (30644, 42.857142857142854),
 (30679, 42.8571

In [76]:
# 테스트 용
print("원글 : ")
print(df["content"][original_index])
print("동의수 : ",df["votes"][original_index])
print("올라온 날짜 : ",df["start"][original_index])
print("--------------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------------")


same_content={}
for index,sum in similar_list[0:11]:
    if(same_content!=df["content"][index]):
        print("유사도 ",round(sum,2),"%")
        print("동의수 : ",df["votes"][index])
        print("올라온 날짜 : ",df["start"][index])
        print(df["content"][index])
        print("--------------------------------------------------------------------------------------------------")
    same_content=df["content"][index]

원글 : 
이명박 前 대통령 출국금지 청원합니다.\n도피성 출국 의도가 다분해보입니다.
동의수 :  0
올라온 날짜 :  2017-11-11 00:00:00
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
유사도  45.45 %
동의수 :  2
올라온 날짜 :  2017-11-11 00:00:00
이명박 전 대통령이란 작자의 출국금지를 청원합니다.지금 청와대 검찰조사로 지난 9년간 사찰 폭압 여론조작 등 온갖 적폐의 근원이자 뿌리가 이명박임이 만천하에 드러났습니다.결코 해외로 출국시켜선 안됩니다.해외 출국해서 무슨말을 할지 언제 귀국해 조사받을지 나가긴 쉬우나 들어오긴 극히 어려울 것입니다.반드시 출국을 막아 이 나라 정치의 모든 적페를 반드시 청산해주시길 바랍니다.
--------------------------------------------------------------------------------------------------
유사도  45.45 %
동의수 :  3
올라온 날짜 :  2017-11-11 00:00:00
이명박 전대통령 출국 금지를 청원합니다.\n외국으로 도피하기전에 출국금지 조치를 해야합니다.
--------------------------------------------------------------------------------------------------
유사도  44.44 %
동의수 :  0
올라온 날짜 :  2017-11-11 00:00:0

In [96]:
# 실제 글쓰기 용
print("원글 : ")
print(original_text)
print("--------------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------------")


same_content={}
for index,sum in similar_list[0:11]:
    if(same_content!=df["content"][index]):
        print("유사도 ",round(sum,2),"%")
        print("동의수 : ",df["votes"][index])
        print("올라온 날짜 : ",df["start"][index])
        print(df["content"][index])
        print("--------------------------------------------------------------------------------------------------")
    same_content=df["content"][index]

원글 : 
이명박 前 대통령 출국금지 청원합니다.\n도피성 출국 의도가 다분해보입니다.
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------
유사도  45.45 %
동의수 :  2
올라온 날짜 :  2017-11-11 00:00:00
이명박 전 대통령이란 작자의 출국금지를 청원합니다.지금 청와대 검찰조사로 지난 9년간 사찰 폭압 여론조작 등 온갖 적폐의 근원이자 뿌리가 이명박임이 만천하에 드러났습니다.결코 해외로 출국시켜선 안됩니다.해외 출국해서 무슨말을 할지 언제 귀국해 조사받을지 나가긴 쉬우나 들어오긴 극히 어려울 것입니다.반드시 출국을 막아 이 나라 정치의 모든 적페를 반드시 청산해주시길 바랍니다.
--------------------------------------------------------------------------------------------------
유사도  45.45 %
동의수 :  3
올라온 날짜 :  2017-11-11 00:00:00
이명박 전대통령 출국 금지를 청원합니다.\n외국으로 도피하기전에 출국금지 조치를 해야합니다.
--------------------------------------------------------------------------------------------------
유사도  44.44 %
동의수 :  0
올라온 날짜 :  2017-11-11 00:00:00
이명박 출국금지 청원합니다!!!\n대통령님 건강하세요~\n청와대 화